In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

wb = gc.open('Compensation_Dashboard_Comp_Review_Data')
sheet = wb.worksheet('Rawdata')

# get_all_values gives a list of rows.
data = sheet.get_all_values()

# Convert to a DataFrame and render.
df = pd.DataFrame(data)

In [ ]:
# CSV 로드시
#from google.colab import drive
#drive.mount('/content/gdrive/')
#df = pd.read_csv("/content/gdrive/My Drive/cp.csv")
#df.info()

In [ ]:
# Data Load
df.columns = df.iloc[0]
df = df.iloc[1:]

<h3> Salary 데이터 전처리

In [ ]:
# 데이터 타입 변환 (Str 로 데이터를 읽어와서 float 로 바꿔줘야 함)
df['Total Base Pay Change Percent'] = df['Total Base Pay Change Percent'].str.replace('%','')
df = df.astype({'Total Base Pay Change Percent': 'float'})

df['STI (Individual) Proposed %'] = df['STI (Individual) Proposed %'].str.replace('%','')
df = df.astype({'STI (Individual) Proposed %': 'float'})

df['LTI Grant Mid 대비 %'] = df['LTI Grant Mid 대비 %'].str.replace('%','')
df = df.astype({'LTI Grant Mid 대비 %': 'float'})

In [ ]:
# Salary 데이터 준비

# People부문 제외
df_salary = df[df["본부"]!="People부문"]

# 보상리뷰 제외자 제외
df_salary = df_salary[df_salary["보상리뷰 제외여부"]=="포함"]

# 승진자 제외여부
# df = df[df["승진여부"] == "승진"]
df_salary = df_salary[df_salary["Promotion Yes/No"]==""]

<H1> Salary Differentiation (Overall)

In [ ]:
plot = go.Figure(data=go.Violin(x=df_salary['2022 Contribution and Impact'],y=df_salary['Total Base Pay Change Percent'],points="all"))
plot.show()

In [ ]:
x0 = df_salary["Total Base Pay Change Percent"][df_salary["2022 Contribution and Impact"] == "Higher"]
x1 = df_salary["Total Base Pay Change Percent"][df_salary["2022 Contribution and Impact"] == "On-Target"]
x2 = df_salary["Total Base Pay Change Percent"][df_salary["2022 Contribution and Impact"] == "Lower"]

fig_sti_hist = go.Figure()
fig_sti_hist.add_trace(go.Histogram(x=x0,name="Higher",nbinsx=20))
fig_sti_hist.add_trace(go.Histogram(x=x1,name="On-Target",nbinsx=20))
fig_sti_hist.add_trace(go.Histogram(x=x2,name="Lower",nbinsx=20))

# Overlay both histograms
fig_sti_hist.update_layout(barmode='overlay')

# Reduce opacity to see both histograms
fig_sti_hist.update_traces(opacity=0.75)
fig_sti_hist.show()

<h3> Salary Differentiation (Engineering)

In [ ]:
# Job Category 선택
df_salary_eng = df_salary[df_salary["Job Category"]=="Engineering"]

# Boxplot
# 본부을 승진 후 Grade 로 바꾸면 Grade 별로 볼 수 있음

fig1 = go.Figure()

fig1.add_trace(go.Box(
    y=df_salary_eng.loc[df_salary_eng["2022 Contribution and Impact"] == "Higher", "Total Base Pay Change Percent"],
    x=df_salary_eng.loc[df_salary_eng["2022 Contribution and Impact"] == "Higher", "본부"],
    name ='Higher',
    marker_color='blue'
))

fig1.add_trace(go.Box(
    y=df_salary_eng.loc[df_salary_eng["2022 Contribution and Impact"] == "On-Target", "Total Base Pay Change Percent"],
    x=df_salary_eng.loc[df_salary_eng["2022 Contribution and Impact"] == "On-Target", "본부"],
    name='On-Target',
    marker_color='green'
))

fig1.add_trace(go.Box(
    y=df_salary_eng.loc[df_salary_eng["2022 Contribution and Impact"] == "Lower", "Total Base Pay Change Percent"],
    x=df_salary_eng.loc[df_salary_eng["2022 Contribution and Impact"] == "Lower", "본부"],
    name='Lower',
    marker_color='red'
))

fig1.update_layout(
    title = "Salary Increase Differentiation (Engineering) ",
    yaxis_title='Total Base Pay Change Percent',
    boxmode='group', # group together boxes of the different traces for each value of x
    height = 800
)
fig1.show()

<h3> Salary Differentiation (Core)

In [ ]:
# Job Category 선택
df_salary_core = df_salary[df_salary["Job Category"]=="Core"]

# Boxplot
fig2 = go.Figure()

fig2.add_trace(go.Box(
    y=df_salary_core.loc[df_salary_core["2022 Contribution and Impact"] == "Higher", "Total Base Pay Change Percent"],
    x=df_salary_core.loc[df_salary_core["2022 Contribution and Impact"] == "Higher", "본부"],
    name ='Higher',
    marker_color='blue'
))

fig2.add_trace(go.Box(
    y=df_salary_core.loc[df_salary_core["2022 Contribution and Impact"] == "On-Target", "Total Base Pay Change Percent"],
    x=df_salary_core.loc[df_salary_core["2022 Contribution and Impact"] == "On-Target", "본부"],
    name='On-Target',
    marker_color='green'
))

fig2.add_trace(go.Box(
    y=df_salary_core.loc[df_salary_core["2022 Contribution and Impact"] == "Lower", "Total Base Pay Change Percent"],
    x=df_salary_core.loc[df_salary_core["2022 Contribution and Impact"] == "Lower", "본부"],
    name='Lower',
    marker_color='red'
))

fig2.update_layout(
    title = "Salary Increase Differentiation (Core) ",
    yaxis_title='Total Base Pay Change Percent',
    boxmode='group', # group together boxes of the different traces for each value of x
    height = 800
)
fig2.show()

<h3> STI 데이터 전처리

In [ ]:
# STI 데이터 준비

# People부문 제외
df_sti= df[df["본부"]!="People부문"]

# 퇴사자 제외 여부
df_sti= df_sti[df_sti["보상리뷰 제외여부"]=="포함"]

# 평가제외자 제외 여부
df_sti = df_sti[df_sti["2022 Contribution and Impact"]!=""]

# 승진자 제외여부
# df_sti= df_sti[df_sti["승진여부"] == "승진"]
#df_sti= df_sti[df_sti["Promotion Yes/No"]==""]

STI Differentiation (Overall)

In [ ]:
viloin_sti = go.Figure(data=go.Violin(x=df_sti["2022 Contribution and Impact"],y=df_sti['STI (Individual) Proposed %'], points="all"))
viloin_sti.show()

STI Histogram

In [ ]:
x0 = df_sti["STI (Individual) Proposed %"][df_sti["2022 Contribution and Impact"] == "Higher"]
x1 = df_sti["STI (Individual) Proposed %"][df_sti["2022 Contribution and Impact"] == "On-Target"]
x2 = df_sti["STI (Individual) Proposed %"][df_sti["2022 Contribution and Impact"] == "Lower"]

fig_sti_hist = go.Figure()
fig_sti_hist.add_trace(go.Histogram(x=x0,name="Higher"))
fig_sti_hist.add_trace(go.Histogram(x=x1,name="On-Target"))
fig_sti_hist.add_trace(go.Histogram(x=x2,name="Lower"))

# Overlay both histograms
fig_sti_hist.update_layout(barmode='overlay')

# Reduce opacity to see both histograms
fig_sti_hist.update_traces(opacity=0.75)
fig_sti_hist.show()

In [ ]:
# boxplot
fig5 = go.Figure()

fig5.add_trace(go.Box(
    y=df_sti.loc[df_sti["2022 Contribution and Impact"] == "Higher", "STI (Individual) Proposed %"],
    x=df_sti.loc[df_sti["2022 Contribution and Impact"] == "Higher", "본부"],
    name ='Higher',
    marker_color='blue'
))

fig5.add_trace(go.Box(
    y=df_sti.loc[df_sti["2022 Contribution and Impact"] == "On-Target", "STI (Individual) Proposed %"],
    x=df_sti.loc[df_sti["2022 Contribution and Impact"] == "On-Target", "본부"],
    name='On-Target',
    marker_color='green'
))

fig5.add_trace(go.Box(
    y=df_sti.loc[df_sti["2022 Contribution and Impact"] == "Lower", "STI (Individual) Proposed %"],
    x=df_sti.loc[df_sti["2022 Contribution and Impact"] == "Lower", "본부"],
    name='Lower',
    marker_color='red'
))

fig5.update_layout(
    title = "STI Differentiation",
    yaxis_title='STI Proposed %',
    boxmode='group', # group together boxes of the different traces for each value of x
    height = 800
)
fig5.show()

<h3> LTI 데이터 전처리

In [ ]:
# LTI 데이터 준비

# People부문 제외
df_lti = df[df["본부"]!="People부문"]

# 퇴사자 제외 여부
df_lti = df_lti[df_lti["보상리뷰 제외여부"]=="포함"]

# 승진자 제외여부
# df_lti = df_lti[df_lti["승진여부"] == "승진"]
df_lti = df_lti[df_lti["Promotion Yes/No"]==""]

# LTI 대상자만
df_lti = df_lti[(df_lti["Job Category"] == "Engineering") | (df_lti["New Compensation Grade For Promotion"]=="25") | (df_lti["New Compensation Grade For Promotion"]=="26") | (df_lti["New Compensation Grade For Promotion"]=="27") | (df_lti["New Compensation Grade For Promotion"]=="28") | (df_lti["New Compensation Grade For Promotion"]=="4B")]


<h3> LTI Differentiation (Overall)

In [ ]:
viloin_lti = go.Figure(data=go.Violin(x=df_lti['2022 Contribution and Impact'],y=df_lti['LTI Grant Mid 대비 %'], points="all"))
viloin_lti.show()

<H3> LTI Differentiation (Engineering)

In [ ]:
# Job Category 선택
df_lti_eng = df_lti[df_lti["Job Category"]=="Engineering"]

# boxplot
fig3 = go.Figure()

fig3.add_trace(go.Box(
    y=df_lti_eng.loc[df_lti_eng["2022 Contribution and Impact"] == "Higher", "LTI Grant Mid 대비 %"],
    x=df_lti_eng.loc[df_lti_eng["2022 Contribution and Impact"] == "Higher", "본부"],
    name ='Higher',
    marker_color='blue'
))

fig3.add_trace(go.Box(
    y=df_lti_eng.loc[df_lti_eng["2022 Contribution and Impact"] == "On-Target", "LTI Grant Mid 대비 %"],
    x=df_lti_eng.loc[df_lti_eng["2022 Contribution and Impact"] == "On-Target", "본부"],
    name='On-Target',
    marker_color='green'
))

fig3.add_trace(go.Box(
    y=df_lti_eng.loc[df_lti_eng["2022 Contribution and Impact"] == "Lower", "LTI Grant Mid 대비 %"],
    x=df_lti_eng.loc[df_lti_eng["2022 Contribution and Impact"] == "Lower", "본부"],
    name='Lower',
    marker_color='red'
))

fig3.update_layout(
    title = "LTI Differentiation",
    yaxis_title='LTI Grant Mid 대비 %',
    boxmode='group', # group together boxes of the different traces for each value of x
    height = 800
)
fig3.show()

<H3> LTI Differentiation (Core)

In [ ]:
#@title
# Job Category 선택
df_lti_core = df_lti[df_lti["Job Category"]=="Core"]

# boxplot
fig4 = go.Figure()

fig4.add_trace(go.Box(
    y=df_lti_core.loc[df_lti_core["2022 Contribution and Impact"] == "Higher", "LTI Grant Mid 대비 %"],
    x=df_lti_core.loc[df_lti_core["2022 Contribution and Impact"] == "Higher", "본부"],
    name ='Higher',
    marker_color='blue'
))

fig4.add_trace(go.Box(
    y=df_lti_core.loc[df_lti_core["2022 Contribution and Impact"] == "On-Target", "LTI Grant Mid 대비 %"],
    x=df_lti_core.loc[df_lti_core["2022 Contribution and Impact"] == "On-Target", "본부"],
    name='On-Target',
    marker_color='green'
))

fig4.add_trace(go.Box(
    y=df_lti_core.loc[df_lti_core["2022 Contribution and Impact"] == "Lower", "LTI Grant Mid 대비 %"],
    x=df_lti_core.loc[df_lti_core["2022 Contribution and Impact"] == "Lower", "본부"],
    name='Lower',
    marker_color='red'
))

fig4.update_layout(
    title = "LTI Differentiation",
    yaxis_title='LTI Grant Mid 대비 %',
    boxmode='group', # group together boxes of the different traces for each value of x
    height = 800
)
fig4.show()